# Module 13 Lesson 4 code: Farmer's problem
## Stochastic programming formulation
### Deterministic model with nominal yields

In [79]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model = ConcreteModel()

# set
I = ['wheat', 'corn', 'sugar beets']

# parameters
# yields, T/acre (nominal value)
# NOTE: it is set mutable for future changes
model.yld = Param(I, mutable=True, initialize={
    'wheat': 2.5,
    'corn': 3,
    'sugar beets': 20})
# planting cost, $/acre
model.c_plant = Param(I, initialize={
    'wheat': 150,
    'corn': 230,
    'sugar beets': 260})
# selling price, $/T (under 6000T for sugar beets)
model.p_sell = Param(I, initialize={
    'wheat': 170,
    'corn': 150,
    'sugar beets': 36})
# selling price for sugar beets above 6000T, $/T
model.p_sell_sb = Param(initialize=10)
# purchase price, $/T
model.p_purc = Param(I, initialize={
    'wheat': 238,
    'corn': 210,
    'sugar beets': 0})
# minimum requirement, T
model.min_demand = Param(I, initialize={
    'wheat': 200,
    'corn': 240,
    'sugar beets': 0})
# total available lands, acres
model.max_land = Param(initialize=500)

# variables
# acres of land devoted to product i
model.x = Var(I, within=NonNegativeReals)
# tons of product i sold
model.y = Var(I, within=NonNegativeReals)
# tons of product i purchased
model.z = Var(I, within=NonNegativeReals)
model.z['sugar beets'].fix(0)
# sales amount of product i
model.s = Var(I, within=NonNegativeReals)

# constraints
# maximum lands
model.con_1 = Constraint(expr=summation(model.x) <= model.max_land)


# minimum requirement for wheat and corn
def con_2(m, i):
    if i == 'sugar beets':
        return Constraint.Skip
    return m.yld[i] * m.x[i] + m.z[i] - m.y[i] >= model.min_demand[i]


model.con_2 = Constraint(I, rule=con_2)
# sales amount
model.con_3_wheat = Constraint(
    expr=model.s['wheat'] == model.p_sell['wheat'] * model.y['wheat'])
model.con_3_corn = Constraint(
    expr=model.s['corn'] == model.p_sell['corn'] * model.y['corn'])
model.con_3_sb_1 = Constraint(
    expr=model.s['sugar beets'] <= model.p_sell['sugar beets'] * model.y[
        'sugar beets'])
model.con_3_sb_2 = Constraint(expr=model.s['sugar beets'] <= model.p_sell_sb * (
            model.y['sugar beets'] - 6000) + model.p_sell['sugar beets'] * 6000)


# sold amount <= produced amount
def con_4(m, i):
    return m.y[i] <= m.yld[i] * m.x[i]


model.con_4 = Constraint(I, rule=con_4)

# objective
model.obj = Objective(
    expr=summation(model.s) - summation(model.c_plant, model.x) - summation(
        model.p_purc, model.z), sense=maximize)

# solve the problem
solver = SolverFactory('glpk')
results = solver.solve(model)
obj_nominal = value(model.obj)

# solving status
print(f"Termination condition: {results.solver.termination_condition}\n")

# print results
print(f"Optimal objective value:\t\t${obj_nominal:,.0f}\n")
print(f"Product\t\t\twheat\tcorn\tsugar beets")
print(f"surface (arces)\t\t"
      f"{value(model.x['wheat']):,.0f}"
      f"\t{value(model.x['corn']):,.0f}"
      f"\t{value(model.x['sugar beets']):,.0f}")
print(f"yield (T)\t\t"
      f"{value(model.x['wheat']) * value(model.yld['wheat']):,.0f}\t"
      f"{value(model.x['corn']) * value(model.yld['corn']):,.0f}\t"
      f"{value(model.x['sugar beets']) * value(model.yld['sugar beets']):,.0f}")
print(f"sells (T)\t\t"
      f"{value(model.y['wheat']):,.0f}"
      f"\t{value(model.y['corn']):,.0f}"
      f"\t{value(model.y['sugar beets']):,.0f}")
print(f"purchase (T)\t\t"
      f"{value(model.z['wheat']):,.0f}"
      f"\t{value(model.z['corn']):,.0f}"
      f"\t{value(model.z['sugar beets']):,.0f}")

Termination condition: optimal

Optimal objective value:		$118,600

Product			wheat	corn	sugar beets
surface (arces)		120	80	300
yield (T)		300	240	6,000
sells (T)		100	0	6,000
purchase (T)		0	0	0


### Actual profit at uncertain yields
#### 1/3 chance with 20% higher yields

In [80]:
# update yield parameter
for i in I:
    model.yld[i] *= 1.2
# fix surface distribution decision from nominal solution
model.x.fix()

# resolve the model
results_higher = solver.solve(model)
obj_higher = value(model.obj)

# solving status
print(f"Termination condition: {results_higher.solver.termination_condition}\n")

# print results
print(f"Optimal objective value:\t\t${obj_higher:,.0f}\n")
print(f"Product\t\t\twheat\tcorn\tsugar beets")
print(f"surface (arces)\t\t"
      f"{value(model.x['wheat']):,.0f}"
      f"\t{value(model.x['corn']):,.0f}"
      f"\t{value(model.x['sugar beets']):,.0f}")
print(f"yield (T)\t\t"
      f"{value(model.x['wheat']) * value(model.yld['wheat']):,.0f}\t"
      f"{value(model.x['corn']) * value(model.yld['corn']):,.0f}\t"
      f"{value(model.x['sugar beets']) * value(model.yld['sugar beets']):,.0f}")
print(f"sells (T)\t\t"
      f"{value(model.y['wheat']):,.0f}"
      f"\t{value(model.y['corn']):,.0f}"
      f"\t{value(model.y['sugar beets']):,.0f}")
print(f"purchase (T)\t\t"
      f"{value(model.z['wheat']):,.0f}"
      f"\t{value(model.z['corn']):,.0f}"
      f"\t{value(model.z['sugar beets']):,.0f}")

Termination condition: optimal

Optimal objective value:		$148,000

Product			wheat	corn	sugar beets
surface (arces)		120	80	300
yield (T)		360	288	7,200
sells (T)		160	48	7,200
purchase (T)		0	0	0


#### 1/3 chance with 20% lower yields

In [81]:
# update yield parameter
for i in I:
    model.yld[i] = model.yld[i] / 1.2 * 0.8
# fix surface distribution decision from nominal solution
model.x.fix()

# resolve the model
results_lower = solver.solve(model)
obj_lower = value(model.obj)

# solving status
print(f"Termination condition: {results_lower.solver.termination_condition}\n")

# print results
print(f"Optimal objective value:\t\t${obj_lower:,.0f}\n")
print(f"Product\t\t\twheat\tcorn\tsugar beets")
print(f"surface (arces)\t\t"
      f"{value(model.x['wheat']):,.0f}"
      f"\t{value(model.x['corn']):,.0f}"
      f"\t{value(model.x['sugar beets']):,.0f}")
print(f"yield (T)\t\t"
      f"{value(model.x['wheat']) * value(model.yld['wheat']):,.0f}\t"
      f"{value(model.x['corn']) * value(model.yld['corn']):,.0f}\t"
      f"{value(model.x['sugar beets']) * value(model.yld['sugar beets']):,.0f}")
print(f"sells (T)\t\t"
      f"{value(model.y['wheat']):,.0f}"
      f"\t{value(model.y['corn']):,.0f}"
      f"\t{value(model.y['sugar beets']):,.0f}")
print(f"purchase (T)\t\t"
      f"{value(model.z['wheat']):,.0f}"
      f"\t{value(model.z['corn']):,.0f}"
      f"\t{value(model.z['sugar beets']):,.0f}")

Termination condition: optimal

Optimal objective value:		$55,120

Product			wheat	corn	sugar beets
surface (arces)		120	80	300
yield (T)		240	192	4,800
sells (T)		40	0	4,800
purchase (T)		0	48	0


#### Average profit

In [87]:
obj_avg = (obj_nominal + obj_lower + obj_higher)/3
print(f"Average objective value:\t\t\t${obj_avg:,.0f}")
print(f"Difference with nominal objective value:\t${obj_avg - obj_nominal:,.0f}")


Average objective value:			$107,240
Difference with nominal objective value:	$-11,360


### Stochastic programming model

In [139]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model_SP = ConcreteModel()

# set
# products
I = ['wheat', 'corn', 'sugar beets']
# scenarios
J = ['nominal', 'lower', 'higher']

# parameters
# yields, T/acre
# define a rule function to initialize yld
yld_nominal = {
    'wheat': 2.5,
    'corn': 3,
    'sugar beets': 20}

def yld_init(m, i, j):
    if j == 'nominal':
        # refer back to nominal model
        return yld_nominal[i]
    elif j == 'lower':
        return yld_nominal[i] * 0.8
    else:
        return yld_nominal[i] * 1.2
model_SP.yld = Param(I, J, initialize=yld_init)
# planting cost, $/acre
model_SP.c_plant = Param(I, initialize={
    'wheat': 150,
    'corn': 230,
    'sugar beets': 260})
# selling price, $/T (under 6000T for sugar beets)
model_SP.p_sell = Param(I, initialize={
    'wheat': 170,
    'corn': 150,
    'sugar beets': 36})
# selling price for sugar beets above 6000T, $/T
model_SP.p_sell_sb = Param(initialize=10)
# purchase price, $/T
model_SP.p_purc = Param(I, initialize={
    'wheat': 238,
    'corn': 210,
    'sugar beets': 0})
# minimum requirement, T
model_SP.min_demand = Param(I, initialize={
    'wheat': 200,
    'corn': 240,
    'sugar beets': 0})
# total available lands, acres
model_SP.max_land = Param(initialize=500)
# probability
model_SP.p = Param(J, initialize={
    'nominal': 1/3,
    'lower': 1/3,
    'higher': 1/3})

# variables
# acres of land devoted to product i
model_SP.x = Var(I, within=NonNegativeReals)
# tons of product i sold
model_SP.y = Var(I, J, within=NonNegativeReals)
# tons of product i purchased
model_SP.z = Var(I, J, within=NonNegativeReals)
model_SP.z['sugar beets', :].fix(0)
# sales amount of product i
model_SP.s = Var(I, J, within=NonNegativeReals)

# constraints
# maximum lands
model_SP.con_1 = Constraint(expr=summation(model_SP.x) <= model_SP.max_land)

# minimum requirement for wheat and corn
def con_2_SP(m, i, j):
    if i == 'sugar beets':
        return Constraint.Skip
    return m.yld[i, j] * m.x[i] + m.z[i, j] - m.y[i, j] >= model_SP.min_demand[i]
model_SP.con_2 = Constraint(I, J, rule=con_2_SP)

# sales amount
def con_3_wheat_SP(m, j):
    return model_SP.s['wheat', j] == model_SP.p_sell['wheat'] * model_SP.y['wheat', j]
model_SP.con_3_wheat = Constraint(J, rule=con_3_wheat_SP)

def con_3_corn_SP(m, j):
    return model_SP.s['corn', j] == model_SP.p_sell['corn'] * model_SP.y['corn', j]
model_SP.con_3_corn = Constraint(J, rule=con_3_corn_SP)

def con_3_sb_1_SP(m, j):
    return model_SP.s['sugar beets', j] <= model_SP.p_sell['sugar beets'] * model_SP.y['sugar beets', j]
model_SP.con_3_sb_1 = Constraint(J, rule=con_3_sb_1_SP)
def con_3_sb_2_SP(m, j):
    return model_SP.s['sugar beets', j] <= model_SP.p_sell_sb * (
            model_SP.y['sugar beets', j] - 6000) + model_SP.p_sell['sugar beets'] * 6000
model_SP.con_3_sb_2 = Constraint(J, rule=con_3_sb_2_SP)

# sold amount <= produced amount
def con_4(m, i, j):
    return m.y[i, j] <= m.yld[i, j] * m.x[i]
model_SP.con_4 = Constraint(I, J, rule=con_4)

# # objective
# model_SP.obj = Objective(
#     expr=summation(model_SP.s, model_SP.p) - summation(model_SP.c_plant, model_SP.x) - summation(summation(model_SP.z, model_SP.p), model_SP.p_purc), sense=maximize)

# objective
model_SP.obj = Objective(expr=sum(model_SP.p[j] * (sum(model_SP.s[i, j] for i in I) - sum(model_SP.p_purc[i] * model_SP.z[i, j] for i in I)) for j in J) - summation(model_SP.c_plant, model_SP.x), sense=maximize)

# solve the problem
solver = SolverFactory('glpk')
results_SP = solver.solve(model_SP)
obj_SP = value(model_SP.obj)

# solving status
print(f"Termination condition: {results_SP.solver.termination_condition}\n")

# print results
print(f"Optimal objective value:\t\t${obj_SP:,.0f}\n")
print(f"Product\t\t\twheat\tcorn\tsugar beets")
print(f"surface (arces)\t\t"
      f"{value(model_SP.x['wheat']):,.0f}"
      f"\t{value(model_SP.x['corn']):,.0f}"
      f"\t{value(model_SP.x['sugar beets']):,.0f}")
print('-'*50)
for j in J:
    print(f"{j} yield")
    print(f"yield (T)\t\t"
          f"{value(model_SP.x['wheat']) * value(model_SP.yld['wheat', j]):,.0f}\t"
          f"{value(model_SP.x['corn']) * value(model_SP.yld['corn', j]):,.0f}\t"
          f"{value(model_SP.x['sugar beets']) * value(model_SP.yld['sugar beets', j]):,.0f}")
    print(f"sells (T)\t\t"
          f"{value(model_SP.y['wheat', j]):,.0f}"
          f"\t{value(model_SP.y['corn', j]):,.0f}"
          f"\t{value(model_SP.y['sugar beets', j]):,.0f}")
    print(f"purchase (T)\t\t"
          f"{value(model_SP.z['wheat', j]):,.0f}"
          f"\t{value(model_SP.z['corn', j]):,.0f}"
          f"\t{value(model_SP.z['sugar beets', j]):,.0f}")
    print(f"profit ($)\t\t\t\t"
          f"{value(sum(model_SP.s[i, j] for i in I) - sum(model_SP.p_purc[i] * model_SP.z[i, j] for i in I) - summation(model_SP.c_plant, model_SP.x)):,.0f}")
    print('-'*50)

Termination condition: optimal

Optimal objective value:		$108,390

Product			wheat	corn	sugar beets
surface (arces)		170	80	250
--------------------------------------------------
nominal yield
yield (T)		425	240	5,000
sells (T)		225	0	5,000
purchase (T)		0	0	0
profit ($)				109,350
--------------------------------------------------
lower yield
yield (T)		340	192	4,000
sells (T)		140	0	4,000
purchase (T)		0	48	0
profit ($)				48,820
--------------------------------------------------
higher yield
yield (T)		510	288	6,000
sells (T)		310	48	6,000
purchase (T)		0	0	0
profit ($)				167,000
--------------------------------------------------


## Robust optimization formulation

In [141]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model_RO = ConcreteModel()

# set
I = ['wheat', 'corn', 'sugar beets']

# parameters
# yields, T/acre (nominal value)
# NOTE: it is set mutable for future changes
model_RO.yld = Param(I, initialize={
    'wheat': 2.5,
    'corn': 3,
    'sugar beets': 20})
# planting cost, $/acre
model_RO.c_plant = Param(I, initialize={
    'wheat': 150,
    'corn': 230,
    'sugar beets': 260})
# selling price, $/T (under 6000T for sugar beets)
model_RO.p_sell = Param(I, initialize={
    'wheat': 170,
    'corn': 150,
    'sugar beets': 36})
# selling price for sugar beets above 6000T, $/T
model_RO.p_sell_sb = Param(initialize=10)
# minimum requirement, T
model_RO.min_demand = Param(I, initialize={
    'wheat': 200,
    'corn': 240,
    'sugar beets': 0})
# total available lands, acres
model_RO.max_land = Param(initialize=500)
# uncertainty range
model_RO.theta = Param(initialize=0.2)

# variables
# acres of land devoted to product i
model_RO.x = Var(I, within=NonNegativeReals)
# tons of product i sold
model_RO.y = Var(I, within=NonNegativeReals)
# sales amount of product i
model_RO.s = Var(I, within=NonNegativeReals)

# constraints
# maximum lands
model_RO.con_1 = Constraint(expr=summation(model_RO.x) <= model_RO.max_land)


# minimum requirement for wheat and corn
def con_2(m, i):
    if i == 'sugar beets':
        return Constraint.Skip
    return (1 - model_RO.theta) * m.yld[i] * m.x[i] - m.y[i] >= model_RO.min_demand[i]


model_RO.con_2 = Constraint(I, rule=con_2)
# sales amount
model_RO.con_3_wheat = Constraint(
    expr=model_RO.s['wheat'] == model_RO.p_sell['wheat'] * model_RO.y['wheat'])
model_RO.con_3_corn = Constraint(
    expr=model_RO.s['corn'] == model_RO.p_sell['corn'] * model_RO.y['corn'])
model_RO.con_3_sb_1 = Constraint(
    expr=model_RO.s['sugar beets'] <= model_RO.p_sell['sugar beets'] *
         model_RO.y['sugar beets'])
model_RO.con_3_sb_2 = Constraint(
    expr=model_RO.s['sugar beets'] <= model_RO.p_sell_sb * (
            model_RO.y['sugar beets'] - 6000) + model_RO.p_sell[
             'sugar beets'] * 6000)


# sold amount <= produced amount
def con_4(m, i):
    return m.y[i] <= (1 - model_RO.theta) * m.yld[i] * m.x[i]


model_RO.con_4 = Constraint(I, rule=con_4)

# objective
model_RO.obj = Objective(
    expr=summation(model_RO.s) - summation(model_RO.c_plant, model_RO.x), sense=maximize)

# solve the problem
solver = SolverFactory('glpk')
results = solver.solve(model_RO)
obj_nominal = value(model_RO.obj)

# solving status
print(f"Termination condition: {results.solver.termination_condition}\n")

# print results
print(f"Optimal objective value:\t\t${obj_nominal:,.0f}\n")
print(f"Product\t\t\twheat\tcorn\tsugar beets")
print(f"surface (arces)\t\t"
      f"{value(model_RO.x['wheat']):,.0f}"
      f"\t{value(model_RO.x['corn']):,.0f}"
      f"\t{value(model_RO.x['sugar beets']):,.0f}")
print(f"yield (T)\t\t"
      f"{value(model_RO.x['wheat']) * (1 - model_RO.theta) * value(model_RO.yld['wheat']):,.0f}\t"
      f"{value(model_RO.x['corn']) * (1 - model_RO.theta) * value(model_RO.yld['corn']):,.0f}\t"
      f"{value(model_RO.x['sugar beets']) * (1 - model_RO.theta) * value(model_RO.yld['sugar beets']):,.0f}")
print(f"sells (T)\t\t"
      f"{value(model_RO.y['wheat']):,.0f}"
      f"\t{value(model_RO.y['corn']):,.0f}"
      f"\t{value(model_RO.y['sugar beets']):,.0f}")


Termination condition: optimal

Optimal objective value:		$56,800

Product			wheat	corn	sugar beets
surface (arces)		100	100	300
yield (T)		200	240	4,800
sells (T)		0	0	4,800
